# Style transfer

#### Setup

In [4]:
import os

try:
    from google.colab import drive
    drive.mount('/content/drive')

    # Change the directory
    os.chdir('/content/drive/MyDrive/ColabNotebooks/style-transfer')
    is_colab = True
    print("Running on Colab")

except:
    is_colab = False
    print("Running locally")

Running locally


In [5]:
# 35s
if is_colab:
    ! pip install pretty_midi
    # ! apt-get install musescore
    ! pip install dfply

    # ! pip install pyfluidsynth
    # ! apt install fluidsynth

    # !cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

In [11]:
#3s
import random
random.seed(42)

import glob
import pandas as pd
from utils.files_utils import save_pickle, load_pickle, data_path, data_tests_path, preprocessed_data_path
from utils.plots_utils import plot_train
from utils.display_audio import save_audios
from model.train import train_model
from model.colab_tension_vae.params import init
from pipeline_tests.test_training import test_reconstruction
from preprocessing import preprocess_data
from tensorflow import keras

## Parámetros

In [10]:
#@title Parámetros ajustables

#@markdown Seleccionar directorios de los datasets
dataset_name = "sonata15_mapleleaf_ds" #@param {type:"string"}
model_name = "urania-test" #@param {type:"string"}
config_name = "4bar" #@param ["8bar", "4bar"]
init(config_name)

#@markdown ¿Entrenamos un nuevo modelo?
if_train = True #@param {type:"boolean"}
verbose = True #@param {type:"boolean"}

#@markdown ¿Cuántas épocas? ¿Cada cuánto grabamos?
epochs =  10#@param {type:"integer"}
checkpt =  1#@param {type:"integer"}

#@markdown Datasets:
dataset1 = "Bach/" #@param {type:"string"}
dataset2 = "ragtime/" #@param {type:"string"}
dataset3 = "Mozart/" #@param {type:"string"}
dataset4 = "Frescobaldi/" #@param {type:"string"}


songs = {
    dataset1[:-1]: [data_path+dataset1+path for path in os.listdir(data_path+dataset1)],
    dataset2[:-1]: [data_path+dataset2+path for path in os.listdir(data_path+dataset2)],
    dataset3[:-1]: [data_path+dataset3+path for path in os.listdir(data_path+dataset3)],
    dataset4[:-1]: [data_path+dataset4+path for path in os.listdir(data_path+dataset4)],
}

In [ ]:
#@title Transformar estilo
ds_original = "Bach" #@param ["Bach", "ragtime", "Mozart", "Frescobaldi"]
ds_objetivo = "Mozart" #@param ["Bach", "ragtime", "Mozart", "Frescobaldi"]

nombre_pickle = ds_original+"2"+ds_objetivo

____
# Preprocessing and Model

Se lo lleva al formato lista de:

- $n$ canciones = lista de:
- 2 voces = lista de:
- $e$ eventos (notas, silencios, etc.)


Armamos el DataFrame con el que manejamos las canciones. Cada fila es una canción y las columnas son:
- `Autor`
- `Título`
- `Id roll`: qué matriz es de la canción
- `Roll` objeto (GuoRoll) con:
    - `matrix`: matriz de $16*n \times 89$ con $n=$ la cantidad de compases
    - `bars`: cantidad de compases por matriz (es el mismo para todo el dataset, con lo cual, podría eliminarse la redundancia en un trabajo futuro)
    - `song`: referencia al objeto `song` al que pertenece si fue obtenido de ahí. Si fue obtenido desde un embedding, es `None` (en un trabajo futuro podría cambiárselo por un singleton).
    - `score`: partitura obtenida a partir de la matriz
    - `midi`: Pretty MIDI obtenido a partir de la matriz

Luego le agregaremos:
- `Embedding`: el vector resultado de encodear el roll

## Preprocessing

In [4]:
try:
    df_preprocessed = load_pickle(file_name=preprocessed_data_path+dataset_name, verbose=verbose)
except:
    print("Processing dataset")
    df_preprocessed = preprocess_data(songs)
    save_pickle(df_preprocessed, file_name=preprocessed_data_path+dataset_name, verbose=verbose)

## Model

In [ ]:
if if_train:
    vae = train_model(
        df=df_preprocessed,
        model_name=model_name,
        final_epoch=epochs,
        ckpt=checkpt,
        verbose=verbose
    )
else:
    vae = keras.models.load_model(data_path + f"saved_models/{model_name}/")

In [ ]:
glob.glob(rf"{data_path}logs/{model_name}_*.csv")

In [ ]:
id_file =  790#@param {type:"integer"}

callbacks = pd.read_csv(data_path + f"logs/{model_name}_{id_file}.csv")
plot_train(callbacks, epochs)

## Test reconstruction

In [ ]:
if is_colab:
  ! sudo apt install lilypond

test_reconstruction(df_preprocessed, vae, model_name, samples=5, inplace=False, verbose=verbose)

# Working space

In [13]:
def save_midis(df, verbose=False):
    rolls_generated = df[df.columns[-1]]
    midis = [r.midi for r in rolls_generated]
    save_audios(df['Titulo'], midis, verbose=verbose)


In [ ]:
df.head()

In [ ]:
for f in os.listdir(f"{data_path}embeddings/brmf_4b"):
    if "df_transferred" in f:
        df = load_pickle(f"{data_path}embeddings/brmf_4b/{f}")
        save_midis(df, verbose=True)